In [1]:
import numpy as np
import scipy as sp

import pandas as pd
#unneded ones are currently included 
from scipy import stats
from scipy.stats import norm, skew #for some statistics
from scipy.special import boxcox1p
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn import preprocessing #for scaling data
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet, Lasso,  Ridge, BayesianRidge, LassoLarsIC
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.inspection import permutation_importance
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

from sklearn.decomposition import PCA


import seaborn as sns

import xgboost as xgb

from astropy.io import fits

from functions1 import *  # own functions


In [2]:
#labels for orientation, mostly ids matter here 
vel=np.loadtxt("velocities_pyas_2templ_all.txt")
print(f"shape of general labels {np.shape(vel)}")
#features
xall=np.loadtxt("shifted_spectra_pyas_2templ_all.txt")
print(f"shape of features {np.shape(xall)}")
#two files with the ids of the stars and label labels in different formats
cal1=np.loadtxt("met_nuc.txt")
print(f"shape of calibrator1 {np.shape(cal1)}")
cal2=np.loadtxt("met_cals.txt")
print(f"shape of calibrator2 {np.shape(cal2)}")

#combine calibrators into same shape, collecting data on ids (can be multiple), S/N and the target label metallicity
calall=np.zeros((37,107))
c=0
for i in range(np.shape(cal1)[1]):
    print(f"star {i}")
    calall[0,c]=1
    calall[1,c]=cal1[0,i]    
    calall[2,c]=cal1[9,i]
    calall[3,c]=cal1[3,i]
    calall[4:6,c]=cal1[1:3,i]
    calall[28,c]=cal1[11,i] #Teff
    calall[29,c]=1 #its source apogee is 1 
    calall[30,c]=cal1[10,i] #logg   
    calall[31,c]=cal1[12,i] #alpha
    calall[32:34,c]= cal1[4:6,i] #coordinates 
    r=np.sqrt((calall[1,c]-vel[4])**2+(calall[4,c]-vel[8])**2+(calall[5,c]-vel[9])**2+(calall[3,c]-vel[10])**2)
    s=np.argmin(r)
    #print(r[s]) 
    calall[6,c]=vel[7,s]
    print(f"s/n of exp 1 is {calall[6,c]}")
    calall[7,c]=s
    c+=1
    #get the ones with multiple data sets
    for j in range(2):
        if cal1[14+2*j,i]!=0:
            calall[0,c-1]+=1
            calall[8+4*j:10+4*j,c-1]=cal1[14+2*j:16+2*j,i]
            r=np.sqrt((cal1[0,i]-vel[4])**2+(vel[8]-cal1[14+2*j,i])**2+(vel[9]-cal1[15+2*j,i])**2+(cal1[3,i]-vel[10])**2)
            s=np.argmin(r)
            #print(r[s])
            calall[10+4*j,c-1]=vel[7,s]
            calall[11+4*j,c-1]=s
            print(f"s/n of exp {j+2} is {calall[10+4*j,c-1]}")
        

print(calall[0:8,0:c].T)   
#now data set 2 
for i in range(np.shape(cal2)[1]):
    print(f"star {i}")
    calall[0,c]=cal2[0,i]
    calall[1,c]=3 #all sources 3   
    calall[2,c]=cal2[36,i]
    calall[3,c]=1 #all are level one
    calall[28,c]=cal2[32,i] #Teff
    calall[29,c]=cal2[35,i] #its source apogee is 1 , 2 Gaia , 3 GES
    calall[30,c]=cal2[31,i] #logg   
    calall[31,c]=cal2[33,i] #alpha
    calall[32:34,c]=cal2[22:24,i]  #coordinates     
    
    for j in range(int(cal2[0,i])):
        calall[4+4*j:6+4*j,c]=cal2[1+2*j:3+2*j,i]
        r=np.sqrt((calall[1,c]-vel[4])**2+(cal2[1+2*j,i]-vel[8])**2+(cal2[2+2*j,i]-vel[9])**2+(calall[3,c]-vel[10])**2)
        s=np.argmin(r)
        print(r[s])
        calall[6+4*j,c]=vel[7,s]
        print(f"s/n of exp {j+1} is {calall[6+4*j,c]}")
        calall[7+4*j,c]=s
    c+=1
    
print(f"number of stars {c}")  
#This file contains the mask and some information used for normalization
resselb=np.loadtxt('norm_mask_v1.txt')

shape of general labels (20, 4748)
shape of features (3072, 4748)
shape of calibrator1 (18, 23)
shape of calibrator2 (37, 84)
star 0
s/n of exp 1 is 84.93251198271109
star 1
s/n of exp 1 is 61.27718963887163
star 2
s/n of exp 1 is 94.28685354637321
star 3
s/n of exp 1 is 82.82941179076919
star 4
s/n of exp 1 is 81.24136913365675
star 5
s/n of exp 1 is 87.03196494035535
s/n of exp 2 is 73.35351937676714
s/n of exp 3 is 78.41684043083046
star 6
s/n of exp 1 is 76.20845331648766
s/n of exp 2 is 89.67876979884284
s/n of exp 3 is 87.5733850498604
star 7
s/n of exp 1 is 45.867832595242
s/n of exp 2 is 72.0095613963618
s/n of exp 3 is 71.07280972798034
star 8
s/n of exp 1 is 47.29908564456481
s/n of exp 2 is 58.60690381083618
s/n of exp 3 is 67.31444730270522
star 9
s/n of exp 1 is 67.59564092137683
s/n of exp 2 is 60.57285497121124
s/n of exp 3 is 62.88290630456617
star 10
s/n of exp 1 is 50.014664089607365
s/n of exp 2 is 66.31461599113013
s/n of exp 3 is 67.10179508782983
star 11
s/n of ex

In [19]:
print(calall.shape)
#print(calall[0:4,:])
# 2 is metallicity #    28 Teff   29 #its source apogee is 1 , 2 Gaia , 3 GES, 30  logg, 31 alpha 32 ra 33 dec
calsel2=np.zeros((10,107))
calsel2[0:2,:]=calall[32:34,:]
calsel2[2,:]=np.where(calall[29,:]==1,1,0)
calsel2[3,:]=np.where(calall[29,:]==2,1,0)
calsel2[4,:]=np.where(calall[29,:]==3,1,0)
calsel2[5,:]=calall[2,:]
calsel2[6,:]=calall[28,:]
calsel2[7:9,:]=calall[30:32,:]
calsel2[9,:]=np.sqrt(calall[6+4*0,:]**2+calall[6+4*1,:]**2+calall[6+4*2,:]**2+calall[6+4*3,:]**2+calall[6+4*5,:]**2)
df = pd.DataFrame(calsel2.T, index=range(107),columns=['ra', 'dec','APOGEE','Gaia','GES','Me','T_eff','logg', 'alpha','S/N'])
df.T_eff=np.where(df.T_eff==0,np.nan,df.T_eff)
df.logg=np.where(df.logg==0,np.nan,df.logg)
df.alpha=np.where(df.alpha==0,np.nan,df.alpha)
print(df.tail())
print(df.describe())

(37, 107)
             ra        dec  APOGEE  Gaia  GES       Me     T_eff      logg  \
102  264.941013 -33.268316     0.0   1.0  0.0  0.03000  4369.000  1.380000   
103  264.954775 -33.266896     0.0   0.0  0.0  0.03000       NaN       NaN   
104  264.931404 -33.235226     0.0   0.0  1.0  0.03000  4510.000  1.840000   
105  264.956092 -33.199196     0.0   0.0  0.0  0.03000       NaN       NaN   
106  266.986768 -24.730966     1.0   0.0  0.0 -0.45975  3297.151  0.025779   

        alpha         S/N  
102 -0.100000  300.616944  
103       NaN  144.023055  
104  7.840000  361.802765  
105       NaN   69.031524  
106  0.045199  184.875349  
               ra         dec      APOGEE        Gaia         GES          Me  \
count  107.000000  107.000000  107.000000  107.000000  107.000000  107.000000   
mean   264.671858  -33.531948    0.560748    0.074766    0.056075   -0.365920   
std      6.395852    6.970650    0.498632    0.264252    0.231149    0.480874   
min    243.310547  -54.228606

Saving the calibration stars. 

In [24]:
dfs=df[df.APOGEE==1]
print(dfs.shape)
df.to_csv('tobias_fritz_calibrators_v1.csv', sep=',')
print(dfs.describe())

(60, 10)
               ra        dec  APOGEE  Gaia   GES         Me        T_eff  \
count   60.000000  60.000000    60.0  60.0  60.0  60.000000    60.000000   
mean   268.684182 -30.396229     1.0   0.0   0.0  -0.189018  3808.945967   
std      2.658154   3.976571     0.0   0.0   0.0   0.417783   345.430568   
min    264.053243 -44.779776     1.0   0.0   0.0  -0.955120  3252.098000   
25%    265.928427 -30.021623     1.0   0.0   0.0  -0.516428  3482.379500   
50%    270.843230 -29.986731     1.0   0.0   0.0  -0.186155  3805.453500   
75%    271.194632 -28.719638     1.0   0.0   0.0   0.193205  4043.693500   
max    271.297736 -24.730966     1.0   0.0   0.0   0.553340  4821.916000   

            logg      alpha         S/N  
count  60.000000  60.000000   60.000000  
mean    1.045769   0.126920  107.328048  
std     0.510285   0.115819   43.497827  
min     0.025779  -0.073935   24.976673  
25%     0.672710   0.019576   81.250620  
50%     1.016891   0.092894  109.956405  
75%     1.36